In [152]:
import geopandas as gpd
import pandas as pd

In [153]:
leases = gpd.read_parquet("our_data/leases_with_nearest_quadrangles.parquet")
leases.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 23117 entries, 0 to 23115
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   OBJECTID            23117 non-null  int64   
 1   landnum             23117 non-null  object  
 2   County              23117 non-null  object  
 3   parcel              23117 non-null  int64   
 4   fundtxt             23117 non-null  object  
 5   open_code           23117 non-null  int64   
 6   leased              23117 non-null  object  
 7   ke                  23117 non-null  object  
 8   lease_              23117 non-null  int64   
 9   lse_code            23117 non-null  int64   
 10  lse_seq             23117 non-null  int64   
 11  effdate             19930 non-null  object  
 12  expdate             19930 non-null  object  
 13  perpetual           0 non-null      float64 
 14  aus                 23117 non-null  float64 
 15  rolodex_            23117 non-nul

In [154]:
from shapely import wkt
wells = pd.read_parquet("our_data/wells_with_depth_trend.parquet")
wells['geometry'] = gpd.points_from_xy(
    x=wells.SITE_LONGIT_DECIMAL, 
    y=wells.SITE_LATITUDE_DECIMAL, 
    crs="EPSG:4326"
    )
wells = gpd.GeoDataFrame(data=wells, geometry='geometry')
wells.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14978 entries, 0 to 14977
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   wellid                 14978 non-null  int64   
 1   SITE_LONGIT_DECIMAL    14978 non-null  float64 
 2   SITE_LATITUDE_DECIMAL  14978 non-null  float64 
 3   depth                  14978 non-null  object  
 4   utc_seconds            14978 non-null  object  
 5   depth_20_year_trend    14978 non-null  float64 
 6   geometry               14978 non-null  geometry
dtypes: float64(3), geometry(1), int64(1), object(2)
memory usage: 819.2+ KB


In [155]:
wells_on_lease = gpd.sjoin(left_df=leases, right_df=wells, how='inner', predicate="contains")
wells_on_lease.groupby("OBJECTID").agg({
    'depth_20_year_trend':'mean',
    })
leases_with_depth_trend = leases.merge(wells_on_lease, on="OBJECTID", suffixes = ["", "_y"])

# leases = gpd.join(left_df=leases, right_df = joined_data, on="OBJECTID")

In [156]:
leases = leases.set_geometry("nearest_quadrangle").to_crs("EPSG:4326")
wells_on_nearest_quadrangle = gpd.sjoin(left_df=leases, right_df=wells, how='inner', predicate="contains")
wells_on_nearest_quadrangle.groupby("OBJECTID").agg({
    'depth_20_year_trend':'mean',
    })
wells_on_nearest_quadrangle = wells_on_nearest_quadrangle.rename(columns ={'depth_20_year_trend':'nearby_depth_20_year_trend'})
leases_with_depth_trend = leases_with_depth_trend.merge(wells_on_nearest_quadrangle, on="OBJECTID", suffixes=["", "_x"])

In [164]:
print(leases_with_depth_trend["depth_20_year_trend"].mean())
print(leases_with_depth_trend["nearby_depth_20_year_trend"].mean())

14.090323716105638
-11.117797746511656
